In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install opencv-python pytesseract pdfplumber pandas transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.2 MB/s eta 0:00:00


Partie Bonus, méthode avec les API Vision

In [14]:
import pandas as pd
import cv2
import pytesseract
import pdfplumber
from typing import List, Tuple
import numpy as np
import os

# Configuration de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Fonction pour extraire les tableaux d'un fichier PDF
def extract_tables_from_pdf(pdf_path: str) -> List[pd.DataFrame]:
    """
    Extrait les tableaux d'un fichier PDF et les retourne sous forme de DataFrames.

    Args:
        pdf_path (str): Chemin vers le fichier PDF.

    Returns:
        List[pd.DataFrame]: Liste de DataFrames représentant les tableaux extraits.
    """
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            print(f"Traitement de la page {page_number + 1}...")

            # Extraire l'image de la page
            image = np.array(page.to_image().original)

            # Convertir en niveaux de gris
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Détection des lignes horizontales et verticales pour les tableaux
            edges = cv2.Canny(gray, 50, 150, apertureSize=3)
            horizontal = cv2.dilate(cv2.erode(edges, np.ones((1, 50))), np.ones((1, 10)))
            vertical = cv2.dilate(cv2.erode(edges, np.ones((50, 1))), np.ones((10, 1)))

            # Fusion des lignes détectées
            grid = cv2.add(horizontal, vertical)

            # Extraire les contours des zones de tableau
            contours, _ = cv2.findContours(grid, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                if w > 50 and h > 20:  # Filtrer les petits blocs
                    roi = image[y:y + h, x:x + w]

                    # OCR sur la région d'intérêt
                    table_text = pytesseract.image_to_string(roi, config="--psm 6")

                    # Structurer les tableaux avec Pandas
                    rows = table_text.split('\n')
                    structured_rows = [row.split() for row in rows if row.strip()]
                    try:
                        df = pd.DataFrame(structured_rows)
                        tables.append(df)
                    except Exception as e:
                        print(f"Erreur lors de la structuration : {e}")

    return tables

# Fonction principale
if __name__ == "__main__":
    pdf_path = "/content/drive/MyDrive/hhhhh/AXA_SA_SFCR_FY22_VF.pdf"  # Remplacez par le chemin de votre fichier PDF

    if os.path.exists(pdf_path):
        print("Fichier trouvé. Début de l'extraction...")
    else:
        raise FileNotFoundError("Fichier non trouvé. Vérifiez le chemin fourni.")

    # Extraire les tableaux
    print("Extraction des tableaux en cours...")
    extracted_tables = extract_tables_from_pdf(pdf_path)

    # Affichage des DataFrames extraits
    print("Voici les tableaux extraits sous forme de DataFrames :\n")
    for i, table in enumerate(extracted_tables):
        print(f"Tableau {i + 1} :")
        print(table)



    print("Traitement terminé.")


Fichier trouvé. Début de l'extraction...
Extraction des tableaux en cours...
Traitement de la page 1...
Traitement de la page 2...
Traitement de la page 3...
Traitement de la page 4...
Traitement de la page 5...
Traitement de la page 6...
Traitement de la page 7...
Traitement de la page 8...
Traitement de la page 9...
Traitement de la page 10...
Traitement de la page 11...
Traitement de la page 12...
Traitement de la page 13...
Traitement de la page 14...
Traitement de la page 15...
Traitement de la page 16...
Traitement de la page 17...
Traitement de la page 18...
Traitement de la page 19...
Traitement de la page 20...
Traitement de la page 21...
Traitement de la page 22...
Traitement de la page 23...
Traitement de la page 24...
Traitement de la page 25...
Traitement de la page 26...
Traitement de la page 27...
Traitement de la page 28...
Traitement de la page 29...
Traitement de la page 30...
Traitement de la page 31...
Traitement de la page 32...
Traitement de la page 33...
Traiteme

Partie Bonus, méthode avec la bibliothèque PdfPlumber

In [7]:
import pdfplumber
import pandas as pd

def extract_tables_with_pdfplumber(pdf_path: str) -> list:
    """
    Extrait les tableaux d'un fichier PDF en utilisant PDFplumber.

    Args:
        pdf_path (str): Chemin vers le fichier PDF.

    Returns:
        list: Liste de DataFrames représentant les tableaux extraits.
    """
    tables = []  # Liste pour stocker les DataFrames

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            print(f"Traitement de la page {page_number + 1}...")
            try:
                # Extraire les tableaux avec PDFplumber
                page_tables = page.extract_tables()

                # Convertir chaque tableau en DataFrame
                for table in page_tables:
                    df = pd.DataFrame(table)  # Crée un DataFrame
                    tables.append((page_number + 1, df))  # Ajouter le DataFrame à la liste
            except Exception as e:
                print(f"Erreur lors de l'extraction de la page {page_number + 1} : {e}")

    return tables  # Retourne la liste des DataFrames

# Fonction principale
if __name__ == "__main__":
    pdf_path = "/content/drive/MyDrive/hhhhh/AXA_SA_SFCR_FY22_VF.pdf"  # Remplacez par le chemin de votre fichier PDF

    if not pdf_path:
        print(f"Erreur : Le fichier {pdf_path} est introuvable.")
    else:
        print("Extraction des tableaux en cours...")
        extracted_tables = extract_tables_with_pdfplumber(pdf_path)

        print(f"\nNombre de tableaux extraits : {len(extracted_tables)}")

        # Affichage des tableaux extraits
        for page_num, table_df in extracted_tables:
            print(f"\nTableau extrait de la page {page_num} :")
            print(table_df)


Extraction des tableaux en cours...
Traitement de la page 1...
Traitement de la page 2...
Traitement de la page 3...
Traitement de la page 4...
Traitement de la page 5...
Traitement de la page 6...
Traitement de la page 7...
Traitement de la page 8...
Traitement de la page 9...
Traitement de la page 10...
Traitement de la page 11...
Traitement de la page 12...
Traitement de la page 13...
Traitement de la page 14...
Traitement de la page 15...
Traitement de la page 16...
Traitement de la page 17...
Traitement de la page 18...
Traitement de la page 19...
Traitement de la page 20...
Traitement de la page 21...
Traitement de la page 22...
Traitement de la page 23...
Traitement de la page 24...
Traitement de la page 25...
Traitement de la page 26...
Traitement de la page 27...
Traitement de la page 28...
Traitement de la page 29...
Traitement de la page 30...
Traitement de la page 31...
Traitement de la page 32...
Traitement de la page 33...
Traitement de la page 34...
Traitement de la page